In [2]:
where = "colab" # "home" # "colab"

import tensorflow as tf
from tensorflow import keras
import os
import math

if where == "colab":
    import pkg_resources
    pkg_resources.require("numpy==1.19.5")  # modified to use specific numpy
    import numpy as np
    from google.colab import drive
    drive.mount('/content/drive')
elif where == "home":
    os.chdir("/home/camilo/anaconda3/envs/tf/lib/python3.7/site-packages")
    import pkg_resources
    pkg_resources.require("numpy==1.19.5")  # modified to use specific numpy
    import numpy as np


tf.config.list_physical_devices('GPU')


Mounted at /content/drive


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
# CONFIGURATION ------ 
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 64  # Number of samples to train on.
total= (num_samples+1)* 3000 #500000
# Path to the data txt file on disk.
save_per_n_epochs= 20

# Dirs 
if where == "colab":
    data_path = "/content/drive/MyDrive/Gutenberg/test_2_half_M.txt"
    checkpoint_filepath= "/content/drive/MyDrive/Gutenberg"
elif where =="home":
    data_path = "/home/camilo/Documents/Own Projects/github/spelll4french/test_2_half_M.txt"
    checkpoint_filepath= "/home/camilo/Documents/Own Projects/Gutenberg"

In [4]:
# INPUT,TARGET LISTS AND PHRASES ------
def init_lists():
  input_texts = []
  target_texts = []
  return(input_texts, target_texts)
 
def append_lines(line,input_list, target_list):
  input_text, target_text = line.split("\t")
  target_text = "\t" + target_text + "\n"
  input_list.append(input_text)
  target_list.append(target_text)
  return(input_text, target_text)
 
def extract_from_lines(line): 
  input_text, target_text = line.split("\t")
  target_text = "\t" + target_text + "\n"
  return(input_text, target_text)
 
# CHARACTER DICTIONNARIES ------
 
def init_dict_training():
  input_characters = set()
  target_characters = set()
  return(input_characters,target_characters)
  
def look_if_in_dict(input_text, target_text,input_dict,target_dict):
  for char in input_text:
    if char not in input_characters:
      input_dict.add(char)
  for char in target_text:
    if char not in target_characters:
      target_dict.add(char)
 
def put_in_dict(input_text, target_text,input_dict,target_dict):
  for char in input_text:
    if char not in input_characters: # probably remove the ifs since we already read all ? 
      input_dict.add(char)
  for char in target_text:
    if char not in target_characters:
      target_dict.add(char)

In [5]:
def get_batch_info():
 
  max_encoder_seq_length = max([len(txt) for txt in input_texts])
  max_decoder_seq_length = max([len(txt) for txt in target_texts])
 
  print("Number of samples:", len(input_texts))
  print("Max sequence length for inputs:", max_encoder_seq_length)
  print("Max sequence length for outputs:", max_decoder_seq_length)
  return(max_encoder_seq_length,max_decoder_seq_length)

In [6]:
# GET ALL UNIQUE CHARACTERS IN A DICT ------
input_characters,target_characters= init_dict_training() # create empty dict to hold characters
max_encoder_seq_length= []
max_decoder_seq_length= []
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
    print('Reading',len(lines), 'lines of the txt file' )
for line in lines[0:len(lines)-1]:
  input_text, target_text= extract_from_lines(line)
  look_if_in_dict(input_text,target_text,input_characters,target_characters)
  max_encoder_seq_length.append(len(input_text))
  max_decoder_seq_length.append(len(target_text))
# Get information about this batch
#max_encoder_seq_length,max_decoder_seq_length= get_batch_info()
 
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
 
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
 
# Dict and index for Input and Output
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

# Reverse-lookup token index to decode sequences back to something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())
 
# Max sentence length 
max_encoder_seq_length= max(max_encoder_seq_length)
max_decoder_seq_length= max(max_decoder_seq_length)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

Reading 502363 lines of the txt file
Number of unique input tokens: 96
Number of unique output tokens: 97
Max sequence length for inputs: 79
Max sequence length for outputs: 73


In [7]:
# MODEL 
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens), name = "Encoder input")
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens), name= "Decoder input")

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
#model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
model = keras.Model(inputs= [encoder_inputs, decoder_inputs],outputs= decoder_outputs)

# callback 
#"{epoch:02d}-{val_accuracy:.2f}.hdf5",
os.chdir(checkpoint_filepath)
model_checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath="{epoch:02d}.hdf5",
    save_weights_only=False,
    monitor='val_accuracy',
    save_freq= 7849*1, #'epoch', #int(0.8*num_samples/batch_size*save_per_n_epochs), # 80% training, 20% val
    save_best_only=False)

model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)

In [8]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Encoder input (InputLayer)      [(None, None, 96)]   0                                            
__________________________________________________________________________________________________
Decoder input (InputLayer)      [(None, None, 97)]   0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 256), (None, 361472      Encoder input[0][0]              
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 256),  362496      Decoder input[0][0]              
                                                                 lstm[0][1]                   

In [9]:
#@title CLASS npyGen
class npyGen(tf.keras.utils.Sequence):   
    def __init__(self, con,
                 batch_size,#,shuffle=True
                 is_val=False, val_split= 0.9):
        self.connection= con 

        self.batch_size = batch_size

        # Validation settings 
        self.is_val= is_val
        self.val_split= val_split
        self.cutoff= math.floor(self.batch_size*self.val_split)
        
        # Shuffle
        #self.shuffle = shuffle # To Do 
        self.n = len(self.connection)

    #def on_epoch_end(self):
     #   if self.shuffle:
     #      self.df.sample(frac=1).reset_index(drop=True)
    
    def __getitem__(self, index):

        input_texts, target_texts= init_lists() # init lists

        for line in lines[index* self.batch_size : (index+1)* self.batch_size ] :
            input_text, target_text= append_lines(line,input_texts, target_texts)

            # Remove "" from strings
            input_texts = [rm.replace('\"', '') for rm in input_texts]
            target_texts = [rm.replace('\"', '') for rm in target_texts]

            encoder_input_data = np.zeros( 
                (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
            )
            decoder_input_data = np.zeros(
                (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
            )
            decoder_target_data = np.zeros(
                (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
            )


        for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
            for t, char in enumerate(input_text):
                encoder_input_data[i, t, input_token_index[char]] = 1.0
            encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
            for t, char in enumerate(target_text):
                # decoder_target_data is ahead of decoder_input_data by one timestep
                decoder_input_data[i, t, target_token_index[char]] = 1.0
                if t > 0:
                    # decoder_target_data will be ahead by one timestep
                    # and will not include the start character.
                    decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
            # Space for everything after
            decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0 
            decoder_target_data[i, t:, target_token_index[" "]] = 1.0

        # Validation split 
        # Either way the output arrays will not be complete

        if not self.is_val: # not val. Then training split
          encoder_input= encoder_input_data[:self.cutoff]
          decoder_input=decoder_input_data[:self.cutoff]
          decoder_target=decoder_target_data[:self.cutoff]
          

        if self.is_val: # if val then val split
          encoder_input= encoder_input_data[self.cutoff:]
          decoder_input=decoder_input_data[self.cutoff:]
          decoder_target=decoder_target_data[self.cutoff:]
          
        return [encoder_input, decoder_input], decoder_target
    
    def __len__(self):
        return self.n // self.batch_size

In [14]:
if where == "colab":
    model= keras.models.load_model('/content/drive/MyDrive/Gutenberg/56.hdf5')
elif where == "home":
    model= keras.models.load_model('/home/camilo/Documents/Own Projects/Gutenberg/Checkpoints/56.hdf5')

In [26]:
# Model FITTING ---
history= model.fit(
npyGen(con= lines,batch_size=64, is_val= False),
#initial_epoch=56,
epochs=5,
validation_data=npyGen(con= lines,batch_size=64, is_val=True),
callbacks=[model_checkpoint_callback])


Epoch 1/5
7849/7849 [==============================] - 523s 67ms/step - loss: 0.2186 - accuracy: 0.9351 - val_loss: 0.2404 - val_accuracy: 0.9295
Epoch 2/5
7849/7849 [==============================] - 555s 71ms/step - loss: 0.2178 - accuracy: 0.9354 - val_loss: 0.2349 - val_accuracy: 0.9313
Epoch 3/5
1338/7849 [====>.........................] - ETA: 4:16 - loss: 0.2139 - accuracy: 0.9364

KeyboardInterrupt: ignored

In [ ]:
#Saving 
name_to_save= "/content/drive/MyDrive/Gutenberg/"+'model_'
model.save(name_to_save)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Encoder input (InputLayer)      [(None, None, 96)]   0                                            
__________________________________________________________________________________________________
Decoder input (InputLayer)      [(None, None, 97)]   0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 256), (None, 361472      Encoder input[0][0]              
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 256),  362496      Decoder input[0][0]              
                                                                 lstm[0][1]                   

In [ ]:
model.layers[2].output

[<KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'lstm')>,
 <KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'lstm')>,
 <KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'lstm')>]

In [ ]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,), name="input_3")
decoder_state_input_c = keras.Input(shape=(latent_dim,), name="input_4")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

In [ ]:
encoder_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Encoder input (InputLayer)   [(None, None, 96)]        0         
_________________________________________________________________
lstm (LSTM)                  [(None, 256), (None, 256) 361472    
Total params: 361,472
Trainable params: 361,472
Non-trainable params: 0
_________________________________________________________________


In [ ]:
decoder_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Decoder input (InputLayer)      [(None, None, 97)]   0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 256)]        0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 256)]        0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 256),  362496      Decoder input[0][0]              
                                                                 input_3[0][0]              

In [ ]:
# Reverse-lookup token index to decode sequences back to
# something readable.
#reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
#reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

In [ ]:
try_input= "parut ni surpris ni contrarié de la trovqr partie il"
# try 
try_correction= np.zeros((1, max_encoder_seq_length, num_encoder_tokens), dtype="float32")
for t, char in enumerate(try_input):
  try_correction[0, t, input_token_index[char]] = 1.0
try_correction[0, t + 1 :, input_token_index[" "]] = 1.0

In [ ]:
try_correction.shape

(1, 79, 96)

In [ ]:
decode_sequence(try_correction)

'parut ni surpris ni contraire de la troppe appelé à\n'

In [ ]:
#pid = os.getpid()
#!kill -9 $pid